# Thuật toán Linear Regression (Hồi quy tuyến tính).
Nội dung:

1. Cài đặt thuật toán.
2. Sử dụng thư viện scikit-learn.
3. Tối ưu hàm mất mát sử dụng thuật toán gradient descent.
4. Mở rộng thuật toán hồi quy tuyến tính. (cập nhật)

1. Cài đặt thuật toán.
 Linear regression là thuật toán supervised, quan hệ đầu vào giữa đầu ra được mô tả bởi một hàm tuyến tính.

In [1]:
import numpy as np
import matplotlib.pyplot as plt # thư viện hiển thị dữ liệu
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
def linear_data(X):
    N,d = X.shape
    one = np.ones((N,1))
    X_bar = np.concatenate((one,X),axis = 1)
    return X_bar
def linear_regression(X_train,y_train):
    X_bar = linear_data(X_train)
    A = np.dot(X_bar.T, X_bar)
    b = np.dot(X_bar.T, y_train)
    w = np.dot(np.linalg.pinv(A), b)
    return w
def linearregression_predic(X_train,y_train,X_test):
    X1 = linear_data(X_test)
    w = linear_regression(X_train,y_train)
    return X1.dot(w)
    

Ví dụ 1. Bài toán về chiều cao và cân nặng.


X_train, y_train: lần lượt là chiều cao và cân nặng tương ứng, X_test là chiều cao mới chưa biết cân nặng, y_pred: là cân nặng dự đoán.

In [3]:

X_train = np.array([[147, 150, 153, 158, 163, 165, 168, 170, 173, 175, 178, 180, 183]]).T
y_train = np.array([ 49, 50, 51,  54, 58, 59, 60, 62, 63, 64, 66, 67, 68])
X_test = np.array([[160,152]]).T
y_pred = linearregression_predic(X_train,y_train,X_test)
print(linear_regression(X_train,y_train))
print(y_pred)

[-33.73541021   0.55920496]
[55.7373837  51.26374401]


2. Sử dụng thư viện scikit-learn.


srouce: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [4]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split # chia dữ liệu thành hai phần test và set
#dữ liệu
X = np.array([[147, 150, 153, 158, 163, 165, 168, 170, 173, 175, 178, 180, 183, 185, 187]]).T
y = np.array([ 49, 50, 51,  54, 58, 59, 60, 62, 63, 64, 66, 67, 68, 71, 72])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 2 ) # lấy 2 điểm dữ liệu dể dự đoán
print(X_train)
print(y_train)


[[180]
 [173]
 [153]
 [183]
 [165]
 [175]
 [147]
 [168]
 [163]
 [150]
 [178]
 [185]
 [170]]
[67 63 51 68 59 64 49 60 58 50 66 71 62]


In [5]:
regr = linear_model.LinearRegression() # khởi tạo mô hình
regr.fit(X_train,y_train)           # luyện mô hình


print("Tham số của mô hình: ")
print(regr.coef_[0], regr.intercept_)
print("Dữ liệu dự đoán:",X_test)
y_pred = regr.predict(X_test)
print("Giá trị dự đoán",y_pred)
print("Gía trị thực:",y_test)


Tham số của mô hình: 
0.5671388336646788 -34.92569582504973
Dữ liệu dự đoán: [[158]
 [187]]
Giá trị dự đoán [54.68223989 71.12926607]
Gía trị thực: [54 72]


3. Tối ưu hàm mất mát bằng thuật toán gradient descent.
Tìm tham số w thông qua thuật toán gradient descent.

Thuật toán linear regression tìm tham số w bằng cách đạo hàm của hàm mất mát và cho bằng 0. Việc giải phương trình đạo hàm bằng 0 gây khó khăn khi điểm dữ liệu có số chiều lớn, thuật toán gradient descent (GD) đơn giản giúp tìm tham số w gần với nghiệm của bài toán cần tìm (khi đạo hàm xấp xỉ 0). 


VD2. Khởi tạo 500 điểm dữ liệu được chọn gần với đường thẳng y = 1 + 2x. Đưa ra tham số của mô hình.

In [6]:
from sklearn.linear_model import LinearRegression
X = np.random.rand(500) # random ra 500 điểm
y = 1 + 2*X + .2* np.random.rand(500)
#print(X)
#print(y)
model = LinearRegression()
X = X.reshape(-1,1) # reshape về ma trận X gồm N hàng 1 cột
y = y.reshape(-1,1)
model.fit(X,y)
w, b = model.coef_[0][0], model.intercept_[0]
sol_sklearn = np.array([b,w])
print("Tham số w giải bằng sklearn",sol_sklearn)
# print(linear_regression(X,y)): test kết quả với phần 1.

Tham số w giải bằng sklearn [1.10318012 2.00169347]


sử dụng GD để tìm tham số w

In [7]:
one = np.ones((X.shape[0],1))
X_bar =np.concatenate((one,X),axis = 1)
# đạo hàm hàm mất mát
def grad(w):
    N = X_bar.shape[0]
    return 1/N*X_bar.T.dot(X_bar.dot(w)-y)

In [8]:
# giá trị hàm mất mát
def cost(w):
    N = X_bar.shape[0]
    return 0.5/N* np.linalg.norm(y-X_bar.dot(w))**2
    

In [9]:
# thuật toán gradient tìm w
def lr_gd(w_init, grad, eta):
    w = [w_init]
    cost_hist =[] # lưu giá trị hàm mất mát sau mỗi vòng lặp
    for it in range(100):
        w_new = w[-1]- eta*grad(w[-1])
        if np.linalg.norm(grad(w_new))/len(w_new) <1e-3:
            break
        cost_hist.append(cost(w_new))
        w.append(w_new)
    return (w, it,cost_hist)


In [10]:
w_init = np.array([[1],[4]])
(w1, it1, cost_hit) =lr_gd(w_init, grad, 1)

print("Kết quả tham số:\n",w1[-1])
print("\nSố vòng lặp",it1)
print("Hàm mất mát:\n",cost_hit)

Kết quả tham số:
 [[1.089225  ]
 [2.02885762]]

Số vòng lặp 59
Hàm mất mát:
 [0.12408308567701776, 0.08757274235988467, 0.07518229493287981, 0.06557191442352428, 0.05727705256825547, 0.050060414153566705, 0.04377834478365891, 0.038309603583440315, 0.03354887754938208, 0.029404502123476492, 0.025796681011036264, 0.02265594884910343, 0.019921833657079547, 0.017541692499583178, 0.015469697893746377, 0.013665955442129446, 0.012095735701884922, 0.010728805500489878, 0.009538845823129723, 0.008502945063677658, 0.007601157882271704, 0.0068161211756877105, 0.00613271976636251, 0.005537795373209033, 0.005019893260717784, 0.0045690416883000535, 0.00417655991336818, 0.00383489105142407, 0.003537456575027188, 0.0032785296501499687, 0.003053124871125913, 0.002856902271135685, 0.002686083760039185, 0.0025373803806390638, 0.002407928982760263, 0.002295237095861821, 0.002197134938752467, 0.0021117336424002144, 0.0020373888814535683, 0.001972669214231934, 0.00191632852160014, 0.0018672820140623364, 0.0